# Validate Monday.com tasks for integration issues

In [2]:
#%load_ext nb_black

import logging

import pandas as pd

import prefect
from prefect import task, Flow, Parameter, unmapped
from prefect.executors import LocalExecutor, LocalDaskExecutor
from prefect.utilities.logging import get_logger

from datetime import timedelta, datetime
from box import Box

from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import validate_task_record, get_items_by_board

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
TASKS_BOARD_ID = "1883170887"

MONDAY_KEY = ""
environment = "dev"

In [4]:
@task
def get_monday_client(key):
    logger = prefect.context.get("logger")
    conn = MondayClient(key)
    logger.info("Monday.com client created.")

    return conn

In [5]:
@task
def get_users(monday_conn):
    logger = prefect.context.get("logger")
    users = monday_conn.users.fetch_users()["data"]["users"]
    users_df = pd.DataFrame(users).set_index("id")
    return users_df

In [6]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

In [7]:
@task
def apply_time_session_breakout(tasks_df):
    logger = prefect.context.get("logger")

    tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
        breakout_time_sessions, axis=1, result_type="expand"
    )
    return tasks_df.reset_index()

In [8]:
@task
def validate_tasks(tasks_df):
    logger = prefect.context.get("logger")

    records = tasks_df.reset_index().to_dict("records")

    vald_recs = []

    for record in records:
        # break the record out into N records
        vald_rec = validate_task_record(record)
        if vald_rec:
            vald_recs.append(vald_rec)

    df = pd.DataFrame(vald_recs).set_index("index")
    return df

In [9]:
@task
def get_tasks(monday_conn, get_only_done=False):

    logger = prefect.context.get("logger")

    if get_only_done:
        tasks_df = get_items_by_board(TASKS_BOARD_ID, "status", "Done")
    else:
        tasks_df = get_items_by_board(monday_conn, TASKS_BOARD_ID)

    # Do not include Posted tasks
    tasks_df = tasks_df.loc[
        ~tasks_df["Integration Message"].str.startswith("Posted", na=False)
    ]

    return tasks_df

In [10]:
@task
def get_df_as_records(df):
    """Helpful for setting up the collection for mapping."""
    logger = prefect.context.get("logger")

    return df.to_dict("records")

In [11]:
@task(max_retries=3, retry_delay=timedelta(seconds=15))
def update_task_integration_status(monday_conn, record):
    logger = prefect.context.get("logger")
    logger.debug(f"Updating Monday.com record for {record['Title']}")
    monday_conn.items.change_item_value(
        TASKS_BOARD_ID,
        record["monday_id"],
        "text01",
        f"{record['integration_state_rule']} - {datetime.now()}",
    )

In [12]:
@task
def display_df(df, title, count=5):
    logger = prefect.context.get("logger")
    logger.info(title)
    display(df.head(count))

In [13]:
with Flow("monday.com task integration") as flow:

    key = Parameter("key")

    conn = get_monday_client(key)

    # get users
    users_df = get_users(conn)

    # get tasks
    task_items_df = get_tasks(conn)
    task_items_df_with_brokedown_time = apply_time_session_breakout(task_items_df)

    # validate against the actual time, session and owner count rules
    validated_tasks_df = validate_tasks(task_items_df_with_brokedown_time)

    # send updates back to Monday.com... this is all one-way so no reduce required
    vald_items = get_df_as_records(validated_tasks_df)
    update_task_integration_status.map(unmapped(conn), vald_items)

In [14]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

params = {"key": MONDAY_KEY}
state = flow.run(parameters=params, executor=LocalDaskExecutor())

[2022-04-24 10:18:22-0600] INFO - prefect.FlowRunner | Beginning Flow run for 'monday.com task integration'
[2022-04-24 10:18:24-0600] INFO - prefect.TaskRunner | Task 'key': Starting task run...
[2022-04-24 10:18:24-0600] INFO - prefect.TaskRunner | Task 'key': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:24-0600] INFO - prefect.TaskRunner | Task 'get_monday_client': Starting task run...
[2022-04-24 10:18:24-0600] INFO - prefect.get_monday_client | Monday.com client created.
[2022-04-24 10:18:24-0600] INFO - prefect.TaskRunner | Task 'get_monday_client': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:24-0600] INFO - prefect.TaskRunner | Task 'get_tasks': Starting task run...
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'get_tasks': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'apply_time_session_breakout': Starting task run...
[2022-04-24 10:18:33-

no_actual_hours_and_no_sessions: extra node page visualizations (beyond 2.0)
no_actual_hours_and_no_sessions: home page infographic
no_actual_hours_and_no_sessions: tools page infographic
no_actual_hours_and_no_sessions: setup automatic product video gifs
no_actual_hours_and_no_sessions: breadcrumbs section
no_actual_hours_and_no_sessions: node page associations section, summary mode
no_actual_hours_and_no_sessions: phenogrid redesign and reimplementation
no_actual_hours_and_no_sessions: node page overall search
no_actual_hours_and_no_sessions: node page associations section, graph mode
no_actual_hours_and_no_sessions: Dremio MVP with GCS
no_actual_hours_and_no_sessions: set up SET bastion proxy, repo
no_actual_hours_and_no_sessions: redesign and implement histo pheno chart
no_actual_hours_and_no_sessions: incorporate 3rd party reactome viewer and genome viewer visualizations
no_actual_hours_and_no_sessions: fix biolink-api dependency issues
no_actual_hours_and_no_sessions: add taxon m

[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'validate_tasks': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'get_df_as_records': Starting task run...
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'get_df_as_records': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'update_task_integration_status': Starting task run...
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'update_task_integration_status': Finished task run for task with final state: 'Mapped'
[2022-04-24 10:18:33-0600] INFO - prefect.TaskRunner | Task 'get_users': Starting task run...
[2022-04-24 10:18:36-0600] INFO - prefect.TaskRunner | Task 'get_users': Finished task run for task with final state: 'Success'
[2022-04-24 10:18:36-0600] INFO - prefect.TaskRunner | Task 'update_task_integration_status[0]': Starting task run...
[2022-04-24 10:18:36-0600]